## Paquetes necesarios

In [16]:
from google.colab import files
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

##Subir Archivos

In [3]:
# Subir archivos
print("🔼 Sube el archivo de inscritos...")
inscritos = files.upload()
nombre_inscritos = list(inscritos.keys())[0]

🔼 Sube el archivo de inscritos...


Saving Inscritos, admitidos y matriculados al 16-04-2025.xlsx to Inscritos, admitidos y matriculados al 16-04-2025.xlsx


In [4]:
print("🔼 Ahora sube el archivo de metas...")
metas = files.upload()
nombre_metas = list(metas.keys())[0]

# Cargar los archivos como excel

xls_inscritos = pd.ExcelFile(nombre_inscritos)
xls_metas = pd.ExcelFile(nombre_metas)

🔼 Ahora sube el archivo de metas...


Saving metas.xlsx to metas.xlsx


## Creación técnica del reporte

## Diccionario Facultad y Entrada

In [5]:
dominio_a_facultad = {
    "Facultad Cs de la Educación": "CIENCIAS DE LA EDUCACIÓN",
    "Facultad Ingeniería": "FACULTAD DE INGENIERÍA",
    "Hábitat, infraestructura y mov": "HABITAT",
    "Identid_educ_cult_com y valor": "IDENTIDADES",
    "Manejo Sosten de Rec Naturales":"MANEJO RECURSOS NATURALES",
    "PUCE Virtual":"PUCE VIRTUAL",
    "Pol eco_comp inst_inn_emp_prod":"POLÍTICA ECONÓMICA",
    "Pol y Der p/part soc_est rel j":"POLÍTICA Y DERECHO",
    "Vida digna_salud int - Cs Salu":"VIDA Y SALUD"
}
facultad_carrera_entrada = {
    ("HABITAT", "Arquitectura"): "HABITAT 1",
    ("HABITAT", "Artes Visuales"): "HABITAT 1",
    ("HABITAT", "Diseño Gráfico"): "HABITAT 1",
    ("HABITAT", "Ingeniería Civil"): "HABITAT 2",
    ("HABITAT", "Sistemas de Información"): "HABITAT 2",

    ("IDENTIDADES", "Comunicación"): "IDENTIDADES 1",
    ("IDENTIDADES", "Educación Básica - Semipres."): "IDENTIDADES 1",
    ("IDENTIDADES", "Pedagogía Musical - Semipresencial"): "IDENTIDADES 1",
    ("IDENTIDADES", "Filosofía"): "IDENTIDADES 2",
    ("IDENTIDADES", "Teología"): "IDENTIDADES 2",

    ("MANEJO RECURSOS NATURALES", "Biología"): "MANEJO RECURSOS NATURALES 1",
    ("MANEJO RECURSOS NATURALES", "Ciencias Biomédicas"): "MANEJO RECURSOS NATURALES 1",

    ("POLÍTICA ECONÓMICA", "Administración de Empresas"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Contabilidad y Auditoría"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Economía"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Finanzas"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Marketing"): "POLÍTICA ECONÓMICA",
    ("POLÍTICA ECONÓMICA", "Negocios Internacionales"): "POLÍTICA ECONÓMICA",

    ("POLÍTICA Y DERECHO", "Derecho SP"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Relaciones Internacionales"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Derecho"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Computación"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Contabilidad y Auditoría"): "POLÍTICA Y DERECHO",
    ("POLÍTICA Y DERECHO", "Administración de Empresas"): "POLÍTICA Y DERECHO",

    ("PUCE VIRTUAL", "Educación Básica"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Educación Inicial"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Ing Ciberseg y Gest Tec Inform"): "PUCE VIRTUAL",
    ("PUCE VIRTUAL", "Ing Redes próx Gener e Inf Nub"): "PUCE VIRTUAL",

    ("VIDA Y SALUD", "Enfermería"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Fisioterapia"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Laboratorio Clínico"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Medicina Veterinaria"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Nutrición y Dietética"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Psicología Clínica"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Psicología Organizacional"): "VIDA Y SALUD 1",
    ("VIDA Y SALUD", "Medicina"): "VIDA Y SALUD 2",
    ("VIDA Y SALUD", "Odontología"): "VIDA Y SALUD 2"
}


In [6]:
# Diccionario de hojas de salida
hojas_final = {}

# Procesar cada hoja
for hoja_inscrito in xls_inscritos.sheet_names:
    print(f"Procesando hoja de inscritos: {hoja_inscrito}")

    # Ajuste para nombre de hoja de metas
    nombre_meta = hoja_inscrito.replace(" 2025-01", "").strip()

    # Leer hojas
    df_base = pd.read_excel(xls_inscritos, sheet_name=hoja_inscrito)
    df_metas = pd.read_excel(xls_metas, sheet_name=nombre_meta)

    # Limpiar columnas
    df_base.columns = df_base.columns.str.strip()
    df_metas.columns = df_metas.columns.str.strip()

    # Renombrar columnas base
    df_base.rename(columns={
        'Facultad_Desc': 'DOMINIO',  # conservar el nombre original como 'DOMINIO'
        'SMRPRLE1.SMRPRLE_PROGRAM_DESC': 'Carrera_Columna',
        'Matriculado ': 'Matriculado'
    }, inplace=True)

    # Crear columna FACULTAD a partir del DOMINIO original
    df_base['FACULTAD'] = df_base['DOMINIO'].map(dominio_a_facultad).fillna(df_base['DOMINIO'])



    if nombre_meta.upper() == 'GRADO':
        # Crear columna FACULTAD con el diccionario (sin tocar DOMINIO original)
        df_base['FACULTAD'] = df_base['DOMINIO'].map(dominio_a_facultad).fillna(df_base['DOMINIO'])

        # Agrupar por FACULTAD y carrera
        agrupado = df_base.groupby(['FACULTAD', 'Carrera_Columna']).agg(
            INSCRITOS=('Carrera_Columna', 'count'),
            ADMITIDOS=('Aceptado_por_institucion', lambda x: (x == 35).sum()),
            MATRICULADOS=('Matriculado', lambda x: (x == 1).sum()),
            RECAUDACIÓN=('Recaudación', 'sum'),
            DOMINIO=('DOMINIO', 'first')  # mantener para cruce con entrada
        ).reset_index()

        # Renombrar Carrera_Columna a CARRERA
        agrupado.rename(columns={'Carrera_Columna': 'CARRERA'}, inplace=True)

        # Hacer merge con las metas
        df_full = pd.merge(agrupado, df_metas, how='left', on=['FACULTAD', 'CARRERA'])

        # Volver a calcular columna ENTRADA después del merge
        df_full['ENTRADA'] = df_full.apply(
            lambda row: facultad_carrera_entrada.get((row['FACULTAD'], row['CARRERA']), ''),
            axis=1
        )

        # Calcular META FACULTAD
        meta_facultad = df_full.groupby('FACULTAD')['META'].sum().reset_index()
        df_full = pd.merge(df_full, meta_facultad, on='FACULTAD', suffixes=('', '_FACULTAD'))
        df_full.rename(columns={'META_FACULTAD': 'META FACULTAD'}, inplace=True)

        # Calcular métricas
        df_full['% CUMPLIMIENTO POR FACULTAD'] = np.where(
            df_full['META FACULTAD'] > 0,
            (df_full['INSCRITOS'] / df_full['META FACULTAD'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        df_full['PROMEDIO PAGO'] = np.where(
            df_full['MATRICULADOS'] > 0,
            df_full['RECAUDACIÓN'] / df_full['MATRICULADOS'],
            0
        )

        df_full['% CUMPLIMIENTO POR CARRERA'] = np.where(
            df_full['META'] > 0,
            (df_full['MATRICULADOS'] / df_full['META'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        df_full['% CUMPLIMIENTO PRESUPUESTO'] = np.where(
            df_full['META PRESUPUESTO'] > 0,
            (df_full['RECAUDACIÓN'] / df_full['META PRESUPUESTO'] * 100).round(0).astype(str) + '%',
            '0%'
        )

        # Orden final de columnas
        columnas_finales = [
            'FACULTAD', 'META FACULTAD', 'ENTRADA', 'CARRERA', 'META',
            'INSCRITOS', 'ADMITIDOS', 'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO POR CARRERA', '% CUMPLIMIENTO POR FACULTAD',
            'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]

        df_final = df_full[columnas_finales]
        hojas_final[hoja_inscrito] = df_final

    else:
        if 'FACULTAD' in df_base.columns:
          df_base.drop(columns=['FACULTAD'], inplace=True)

        df_base.rename(columns={
            'DOMINIO': 'FACULTAD'
        }, inplace=True)

        # Definir la columna clave de carrera según el tipo de hoja
        carrera_key = 'TECNOLOGÍA' if nombre_meta.upper() == 'PUCETEC' else 'CARRERA'

        # Agrupar usando DOMINIO original (sin aplicar el diccionario)
        agrupado = df_base.groupby(['FACULTAD', 'Carrera_Columna']).agg(
            INSCRITOS=('Carrera_Columna', 'count'),
            ADMITIDOS=('Aceptado_por_institucion', lambda x: (x == 35).sum()),
            MATRICULADOS=('Matriculado', lambda x: (x == 1).sum()),
            RECAUDACIÓN=('Recaudación', 'sum')
        ).reset_index()

        # Renombrar Carrera_Columna a su nombre correspondiente (CARRERA o TECNOLOGÍA)
        agrupado.rename(columns={'Carrera_Columna': carrera_key}, inplace=True)

        # Validar columna en metas
        if carrera_key not in df_metas.columns:
            raise KeyError(f"La hoja de metas '{nombre_meta}' no tiene la columna '{carrera_key}'")

        df_metas = df_metas.drop_duplicates(subset=[carrera_key])
        df_full = pd.merge(agrupado, df_metas, how='left', on=carrera_key)

        # Calcular métricas
        df_full['PROMEDIO PAGO'] = np.where(df_full['MATRICULADOS'] > 0,
                                            df_full['RECAUDACIÓN'] / df_full['MATRICULADOS'], 0)
        df_full['% CUMPLIMIENTO POR CARRERA'] = np.where(df_full['META'] > 0,
                                                        (df_full['MATRICULADOS'] / df_full['META'] * 100).round(0).astype(str) + '%',
                                                        '0%')
        df_full['% CUMPLIMIENTO PRESUPUESTO'] = np.where(df_full['META PRESUPUESTO'] > 0,
                                                        (df_full['RECAUDACIÓN'] / df_full['META PRESUPUESTO'] * 100).round(0).astype(str) + '%',
                                                        '0%')

        columnas_finales = [
            'FACULTAD', carrera_key, 'META', 'INSCRITOS', 'ADMITIDOS',
            'MATRICULADOS', 'RECAUDACIÓN', 'PROMEDIO PAGO',
            '% CUMPLIMIENTO POR CARRERA', 'META PRESUPUESTO', '% CUMPLIMIENTO PRESUPUESTO'
        ]

        if nombre_meta.upper() == 'PUCETEC':
            columnas_finales.remove('FACULTAD')

        df_final = df_full[columnas_finales]
        hojas_final[hoja_inscrito] = df_final


# Guardar Excel final
nombre_final = "REPORTE_MATRICULADOS_SIN_FORMATO.xlsx"
with pd.ExcelWriter(nombre_final, engine='openpyxl') as writer:
    for hoja, df in hojas_final.items():
        df.to_excel(writer, sheet_name=hoja, index=False)

files.download(nombre_final)

Procesando hoja de inscritos: GRADO 2025-01
Procesando hoja de inscritos: PUCETEC 2025-01
Procesando hoja de inscritos: POSGRADO HÍBRIDO 2025-01
Procesando hoja de inscritos: POSGRADO VIRTUAL 2025-01


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Formato del Reporte

In [15]:
# Cargar archivo fuente
ruta_archivo = "REPORTE_MATRICULADOS_SIN_FORMATO.xlsx"
wb = openpyxl.load_workbook(ruta_archivo)

# Función para combinar y centrar valores repetidos
def combinar_y_centrar(sheet, col_name, header_row=2):
    headers = [cell.value for cell in sheet[header_row]]
    if col_name not in headers:
        return
    col_idx = headers.index(col_name) + 1
    prev_value, start_row = None, None
    for row in range(header_row + 1, sheet.max_row + 1):
        value = sheet.cell(row=row, column=col_idx).value
        if value != prev_value:
            if start_row and row - start_row > 1:
                sheet.merge_cells(start_row=start_row, start_column=col_idx, end_row=row-1, end_column=col_idx)
                sheet.cell(start_row, col_idx).alignment = Alignment(horizontal='center', vertical='center')
            start_row, prev_value = row, value
    if start_row and sheet.max_row - start_row >= 1:
        sheet.merge_cells(start_row=start_row, start_column=col_idx, end_row=sheet.max_row, end_column=col_idx)
        sheet.cell(start_row, col_idx).alignment = Alignment(horizontal='center', vertical='center')

# Función para agregar fila TOTAL con valores únicos de META FACULTAD
def agregar_fila_total_unicos(df, hoja_nombre):
    headers = df.columns.tolist()
    total_row = {}

    for col in headers:
        if col == 'PROMEDIO PAGO':
            total_row[col] = df['RECAUDACIÓN'].sum() / df['MATRICULADOS'].sum() if df['MATRICULADOS'].sum() > 0 else 0
        elif isinstance(df[col].iloc[0], str) and '%' in df[col].iloc[0]:
            total_row[col] = ''
        elif pd.api.types.is_numeric_dtype(df[col]):
            if col == 'META FACULTAD':
                total_row[col] = df[['FACULTAD', col]].drop_duplicates()[col].sum()
            else:
                total_row[col] = df[col].sum()
        else:
            total_row[col] = ''

    total_row[headers[0]] = 'TOTAL'

    # Porcentajes
    if 'META' in df.columns and 'MATRICULADOS' in df.columns and df['META'].sum() > 0:
        total_row['% CUMPLIMIENTO POR CARRERA'] = f"{(df['MATRICULADOS'].sum() / df['META'].sum() * 100):.0f}%"
    if hoja_nombre.upper().startswith("GRADO") and 'META FACULTAD' in df.columns:
        suma_mf = df[['FACULTAD', 'META FACULTAD']].drop_duplicates()['META FACULTAD'].sum()
        if suma_mf > 0:
            total_row['% CUMPLIMIENTO POR FACULTAD'] = f"{(df['INSCRITOS'].sum() / suma_mf * 100):.0f}%"
    if 'META PRESUPUESTO' in df.columns and df['META PRESUPUESTO'].sum() > 0:
        total_row['% CUMPLIMIENTO PRESUPUESTO'] = f"{(df['RECAUDACIÓN'].sum() / df['META PRESUPUESTO'].sum() * 100):.0f}%"

    return pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)

# Función principal por hoja
def procesar_hoja(sheet):
    hoja_nombre = sheet.title
    headers = [cell.value for cell in sheet[1]]
    data = [
        [cell.value for cell in row]
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row)
        if any(cell.value is not None for cell in row)
    ]
    if not data:
        return

    df = pd.DataFrame(data, columns=headers)

    # Ordenar si existen
    ordenar_por = [col for col in ['FACULTAD', 'ENTRADA'] if col in df.columns]
    if ordenar_por:
        df = df.sort_values(by=ordenar_por)

    # Eliminar fila TOTAL si ya existía
    df = df[df[headers[0]] != 'TOTAL']

    # Agregar fila TOTAL con lógica especial
    df = agregar_fila_total_unicos(df, hoja_nombre)

    # Limpiar y reescribir
    sheet.delete_rows(1, sheet.max_row)

    # Insertar título
    sheet.insert_rows(1)
    sheet.cell(row=1, column=1).value = f"SEGUIMIENTO {hoja_nombre}"
    sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(headers))
    sheet.cell(row=1, column=1).alignment = Alignment(horizontal='center', vertical='center')

    # Encabezados
    for col_idx, header in enumerate(headers, start=1):
        sheet.cell(row=2, column=col_idx, value=header)

    # Escribir filas
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=3):
        for c_idx, value in enumerate(row, start=1):
            sheet.cell(row=r_idx, column=c_idx, value=value)

    # Combinar columnas clave
    for col in ['FACULTAD', 'META FACULTAD', 'ENTRADA']:
        combinar_y_centrar(sheet, col_name=col, header_row=2)

# Procesar todas las hojas
for hoja in wb.sheetnames:
    procesar_hoja(wb[hoja])

# Guardar
wb.save("REPORTE_MATRICULADOS_FINAL.xlsx")
files.download("REPORTE_MATRICULADOS_FINAL.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>